In [1]:
#importing the required packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [2]:
a=pd.read_csv('train.csv') # 'a' is the training set's dataframe

In [3]:
a = pd.get_dummies(a, columns=['stored+AF8-flag'], drop_first=True) #Changed the stored flag value from Y and N to 1 and 0
a.rename(columns={'pickup+AF8-time':'pickuptime'},inplace=True)
a.rename(columns={'drop+AF8-time':'droptime'},inplace=True)
a.head()

,ID,vendor+AF8-id,pickup+AF8-loc,drop+AF8-loc,driver+AF8-tip,mta+AF8-tax,distance,pickuptime,droptime,num+AF8-passengers,toll+AF8-amount,payment+AF8-method,rate+AF8-code,extra+AF8-charges,improvement+AF8-charge,total+AF8-amount,stored+AF8-flag_Y
0,0,1,170,233,1.83,0.5,0.70,04/04/2017 05:59:43 PM,04/04/2017 06:05:04 PM,1,0.0,1,1,1.0,0.3,9.13,0
1,1,2,151,243,3.56,0.5,4.64,04/03/2017 07:03:34 PM,04/03/2017 07:20:04 PM,1,0.0,1,1,1.0,0.3,21.36,0
2,2,2,68,90,1.50,0.5,1.29,04/03/2017 03:06:13 PM,04/03/2017 03:12:30 PM,2,0.0,1,1,0.0,0.3,8.80,0
3,3,2,142,234,1.50,0.5,2.74,04/04/2017 08:10:52 AM,04/04/2017 08:27:00 AM,1,0.0,1,1,0.0,0.3,14.80,0
4,4,2,238,238,0.00,0.5,0.45,04/05/2017 02:02:59 PM,04/05/2017 02:05:41 PM,6,0.0,2,1,0.0,0.3,4.80,0


In [4]:
a['pickuptime'] = pd.to_datetime(a.pickuptime) #processing the  object type data to datetime for easier calculation



In [5]:
a['droptime'] = pd.to_datetime(a.droptime) #




In [6]:
duration=(a['droptime']-a['pickuptime']) #calculating the duration for which the chh-ola was booked 
a.drop(columns=['pickuptime', 'droptime',],inplace=True) #dropped the column pickup and droptime as it is not required
a.head()

,ID,vendor+AF8-id,pickup+AF8-loc,drop+AF8-loc,driver+AF8-tip,mta+AF8-tax,distance,num+AF8-passengers,toll+AF8-amount,payment+AF8-method,rate+AF8-code,extra+AF8-charges,improvement+AF8-charge,total+AF8-amount,stored+AF8-flag_Y
0,0,1,170,233,1.83,0.5,0.70,1,0.0,1,1,1.0,0.3,9.13,0
1,1,2,151,243,3.56,0.5,4.64,1,0.0,1,1,1.0,0.3,21.36,0
2,2,2,68,90,1.50,0.5,1.29,2,0.0,1,1,0.0,0.3,8.80,0
3,3,2,142,234,1.50,0.5,2.74,1,0.0,1,1,0.0,0.3,14.80,0
4,4,2,238,238,0.00,0.5,0.45,6,0.0,2,1,0.0,0.3,4.80,0


In [7]:
timeinsec=duration.dt.total_seconds() #converting the datetime to time in seconds
a['timeinsec']=timeinsec
a.loc[a['ID'] >=0, 'ID'] = 1.0 #making theta(0)th column by all values to 1

In [8]:
convert_dict = {'vendor+AF8-id':float,'driver+AF8-tip':float,'ID':float,'mta+AF8-tax':float,
                'toll+AF8-amount':float,
               'extra+AF8-charges':float,'improvement+AF8-charge':float,'total+AF8-amount':float}
a=a.astype(convert_dict) #converted all column values to float type for calculation

In [9]:
y= a['total+AF8-amount'] #separated the price values from the dataframe a
a.drop(columns=['total+AF8-amount'],inplace=True) #dropped the y values from the dataframe a
a=a.as_matrix(columns=None) #converted the dataframe to matrix 

/home/rishabh007/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
#theta = inverse(transpose(a)*a)*transpose(a)*y 
#y is price vector
#this is the normal equation to find theta vector
at=np.transpose(a); #at is a transpose
at

array([[1.00e+00, 1.00e+00, 1.00e+00, ..., 1.00e+00, 1.00e+00, 1.00e+00],
       [1.00e+00, 2.00e+00, 2.00e+00, ..., 1.00e+00, 2.00e+00, 1.00e+00],
       [1.70e+02, 1.51e+02, 6.80e+01, ..., 1.64e+02, 2.64e+02, 7.40e+01],
       ...,
       [3.00e-01, 3.00e-01, 3.00e-01, ..., 3.00e-01, 3.00e-01, 3.00e-01],
       [0.00e+00, 0.00e+00, 0.00e+00, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [3.21e+02, 9.90e+02, 3.77e+02, ..., 8.02e+02, 7.67e+02, 1.66e+02]])

In [11]:
ata=at.dot(a) #transpose(a)*a
ata

array([[1.04857300e+06, 1.61633900e+06, 1.72813689e+08, 1.70568813e+08,
        1.98727497e+06, 5.21652000e+05, 2.99874437e+06, 1.66796100e+06,
        3.62652080e+05, 1.38195900e+06, 1.09404800e+06, 3.58105350e+05,
        3.14486700e+05, 7.16900000e+03, 1.01694591e+09],
       [1.61633900e+06, 2.75187100e+06, 2.65783071e+08, 2.62488530e+08,
        3.08181492e+06, 8.04387500e+05, 4.66248354e+06, 2.76362600e+06,
        5.66329550e+05, 2.12502700e+06, 1.68615800e+06, 5.54087200e+05,
        4.84757700e+05, 7.16900000e+03, 1.62493222e+09],
       [1.72813689e+08, 2.65783071e+08, 3.30201090e+10, 2.86467577e+10,
        3.19845214e+08, 8.59723405e+07, 4.70766997e+08, 2.74324240e+08,
        5.49918304e+07, 2.27459224e+08, 1.80115990e+08, 5.86715791e+07,
        5.18284017e+07, 1.15653200e+06, 1.63956241e+11],
       [1.70568813e+08, 2.62488530e+08, 2.86467577e+10, 3.28153599e+10,
        3.12509528e+08, 8.49780065e+07, 4.59537607e+08, 2.70859546e+08,
        5.23833875e+07, 2.24590451e+0

In [12]:
atainv = np.linalg.inv(ata) #inverse(transpose(a)*a)

In [13]:
ans=atainv.dot(at)  #inverse(transpose(a)*a)*transpose(a)*y 

In [14]:
ans

array([[ 3.78147288e-06, -1.61844146e-07,  4.57656745e-06, ...,
         7.79022209e-06, -1.28625341e-05,  3.32352077e-06],
       [-2.01923111e-06,  2.18614683e-06,  1.66302111e-06, ...,
        -2.06584636e-06,  8.83943130e-07, -1.98027230e-06],
       [-1.51108868e-09, -4.16984698e-09, -2.05706962e-08, ...,
         6.62931693e-10,  2.01567687e-08, -1.92860878e-08],
       ...,
       [ 4.83856817e-06, -4.59102899e-06,  6.86779740e-06, ...,
        -1.41777510e-06,  3.93742228e-06,  3.07053785e-06],
       [-1.71592950e-06, -3.31736760e-09,  2.43459514e-07, ...,
        -1.88450365e-06,  9.94842319e-08, -1.99435851e-06],
       [-1.66341907e-11, -3.37993808e-11, -4.41797241e-11, ...,
         1.29333841e-11, -1.24654669e-11, -3.43015646e-11]])

In [15]:
theta=ans.dot(y) #theta
theta

array([ 7.75263259e+00, -3.88020169e-03,  5.31206703e-04,  7.45182196e-04,
        1.50859331e+00, -4.02810075e+01,  2.47484662e+00,  3.94025844e-03,
        9.64667669e-01,  9.45928186e-01,  1.23357193e+00,  5.45091071e-01,
        5.07182540e+01,  3.74812934e-01,  1.03731107e-04])

In [16]:
b=pd.read_csv('test.csv') #reading the test data

In [17]:
b = pd.get_dummies(b, columns=['stored_flag'], drop_first=True)
b.rename(columns={'pickup_time':'pickuptime'},inplace=True)
b.rename(columns={'drop_time':'droptime'},inplace=True)


In [18]:
b['pickuptime'] = pd.to_datetime(b.pickuptime)
b['droptime'] = pd.to_datetime(b.droptime)


In [19]:
duration1=(b['droptime']-b['pickuptime'])
b.drop(columns=['pickuptime', 'droptime',],inplace=True)

In [20]:
b.head()

,ID,vendor_id,pickup_loc,drop_loc,driver_tip,mta_tax,distance,num_passengers,toll_amount,payment_method,rate_code,extra_charges,improvement_charge,stored_flag_Y
0,0,1,68.0,186.0,1.00,0.5,0.70,1.0,0.0,1.0,1.0,0.5,0.3,0
1,1,1,170.0,161.0,2.55,0.5,1.00,1.0,0.0,1.0,1.0,0.0,0.3,0
2,2,2,13.0,246.0,2.96,0.5,3.35,2.0,0.0,1.0,1.0,0.0,0.3,0
3,3,1,170.0,233.0,2.00,0.5,0.80,2.0,0.0,1.0,1.0,0.0,0.3,0
4,4,1,237.0,236.0,1.00,0.5,0.70,1.0,0.0,1.0,1.0,0.0,0.3,1


In [21]:
timeinsec=duration1.dt.total_seconds()
b['timeinsec']=timeinsec
b.loc[b['ID'] >=0, 'ID'] = 1.0

In [22]:
convert_dict = {'vendor_id':float}
b=b.astype(convert_dict)


In [23]:
b=b.as_matrix(columns=None)

/home/rishabh007/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [24]:
h=b.dot(theta)
h.shape

(154235,)

In [25]:
r,c =b.shape
print (r,c)

154235 15


In [26]:
array = np.arange(r)
array.shape


(154235,)

In [27]:
dataset = pd.DataFrame({'total_amount': h[:]})

In [28]:
dataset.shape


(154235, 1)

In [29]:
dataset.to_csv('ans.csv') 

In [30]:
dataset.head()

,total_amount
0,8.722082
1,11.654657
2,18.046146
3,10.308097
4,8.967564
